In [1]:
import pandas as pd

In [2]:
# df sales

sales = pd.read_csv('sales.csv')
sales

,product_id,ordered_at,quantity_ordered
0,3998909,9/18/18 17:51,1
1,3998909,9/18/18 12:52,1
2,3998909,9/18/18 11:33,1
3,3998909,9/18/18 18:47,1
4,3998909,9/18/18 17:36,1
...,...,...,...
170,4085861,9/13/18 23:29,1
171,64,9/11/18 19:32,1
172,64,9/14/18 20:10,1
173,64,9/12/18 16:00,1


In [3]:
# Check null in sales
sales.isna().sum()

product_id          0
ordered_at          0
quantity_ordered    0
dtype: int64

In [4]:
#df prices

prices = pd.read_csv('prices.csv')
prices

,product_id,old_price,new_price,updated_at
0,64,270000,239000,9/10/18 16:37
1,3954203,60000,64000,9/11/18 11:54
2,3954203,60500,57500,9/17/18 22:59
3,3954203,64000,60500,9/15/18 3:49
4,3954203,68800,60000,9/10/18 16:32
5,3998909,15500,16500,9/16/18 5:09
6,3998909,17000,15500,9/13/18 6:43
7,3998909,19000,17000,9/10/18 16:35
8,4085861,53500,52000,9/17/18 22:59
9,4085861,53500,67000,9/12/18 3:51


In [5]:
prices.shape

(14, 4)

In [6]:
# Check null in prices
prices.isna().sum()

product_id    0
old_price     0
new_price     0
updated_at    0
dtype: int64

In [7]:
# Merging sales and prices as SP

In [8]:
## convert ordered_at, updated_at to datetime object
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
prices['updated_at'] = pd.to_datetime(prices['updated_at'])


In [9]:
## use merge_asof to merge sales and prices as SP 
SP = pd.merge_asof(sales.sort_values('ordered_at'), prices.sort_values('updated_at'), by='product_id', left_on='ordered_at', right_on='updated_at',direction='nearest')
SP.head()

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at
0,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00
1,4085861,2018-09-11 06:26:00,1,60000,53500,2018-09-11 08:51:00
2,4085861,2018-09-11 06:53:00,1,60000,53500,2018-09-11 08:51:00
3,4085861,2018-09-11 08:24:00,1,60000,53500,2018-09-11 08:51:00
4,4085861,2018-09-11 09:30:00,1,60000,53500,2018-09-11 08:51:00


In [10]:
SP.shape

(175, 6)

In [11]:
# create New_price_rev column
SP_new = SP[SP['ordered_at'] > SP['updated_at']]
SP['new_price_rev'] = SP_new['new_price'] * SP_new['quantity_ordered']

In [12]:
# create old_price_rev column
SP_old = SP[SP['ordered_at'] < SP['updated_at']]
SP['old_price_rev'] = SP_old['old_price'] * SP_old['quantity_ordered']

SP.head()

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at,new_price_rev,old_price_rev
0,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00,60000.0,NaN
1,4085861,2018-09-11 06:26:00,1,60000,53500,2018-09-11 08:51:00,NaN,60000.0
2,4085861,2018-09-11 06:53:00,1,60000,53500,2018-09-11 08:51:00,NaN,60000.0
3,4085861,2018-09-11 08:24:00,1,60000,53500,2018-09-11 08:51:00,NaN,60000.0
4,4085861,2018-09-11 09:30:00,1,60000,53500,2018-09-11 08:51:00,53500.0,NaN


In [13]:
# Group product id and sum
SP_by_id_type = SP.groupby('product_id')['new_price_rev', 'old_price_rev'].sum()
print(SP_by_id_type)

            new_price_rev  old_price_rev
product_id                              
64               956000.0            0.0
3954203          757500.0       120000.0
3998909          246500.0        34000.0
4085861         4971000.0      3276500.0


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15648\3474619051.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  SP_by_id_type = SP.groupby('product_id')['new_price_rev', 'old_price_rev'].sum()


In [14]:
# total_revenue = new_price_rev + old_price_rev
SP_by_id_type['total_revenue'] = SP_by_id_type['new_price_rev'] + SP_by_id_type['old_price_rev'] 

In [15]:
# drop new_price_rev and old_price_rev
revenue_by_product = SP_by_id_type.drop(columns=['new_price_rev', 'old_price_rev'])
revenue_by_product

,total_revenue
product_id,
64,956000.0
3954203,877500.0
3998909,280500.0
4085861,8247500.0


In [18]:
# export to csv file
# revenue_by_product.to_csv('revenue_by_product.csv')